In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
df = pd.read_csv('wine_data.csv')
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [23]:
#data = data[["country", "points", "province", "variety", "winery", "price"]]
df = df[["country", "points", "province", "variety", "price"]]

In [27]:
df = df.loc[(df['country'].isna() == False) & (df['points'].isna() == False) & \
            (df['variety'].isna() == False) & (df['province'].isna() == False)].copy()

df.count()

country          129772
points           129772
province         129772
variety          129772
price            120789
variety_count    129772
dtype: int64

In [28]:
df['variety_count']= df.variety.map(df.variety.value_counts())
df = df[(df.variety_count != 1)]


In [29]:
df.drop(df.loc[df['price']>500].index, inplace=True)

In [30]:
data_new = pd.get_dummies(df)
# data_new is now encoded using dummy variables
data_new.head()


,points,price,variety_count,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Bosnia and Herzegovina,country_Brazil,country_Bulgaria,...,variety_Xinomavro,variety_Xynisteri,variety_Yapincak,variety_Zibibbo,variety_Zierfandler,variety_Zierfandler-Rotgipfler,variety_Zinfandel,variety_Zlahtina,variety_Zweigelt,variety_Çalkarası
0,87,NaN,2354,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,87,15.0,2466,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,87,14.0,1452,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,87,13.0,5188,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,87,65.0,13269,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
hasprice = data_new.loc[data_new['price'].isna() == False].copy()
noprice = data_new.loc[data_new['price'].isna()]

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
y = hasprice["price"]
X = hasprice.drop(columns=["price"])
y = y.values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [33]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [ ]:
'''import matplotlib.pyplot as plt
hasprice.hist(bins=20, figsize=(20,15))
plt.show()'''

In [34]:
#for no price data; "predictions" replace X_test_scaled with new data w/o prices
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)
predictions = model.predict(X_test_scaled)

In [36]:
from sklearn.liear_model import Lasso
from sklearn.metrics import mean_squared_error
lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)
predictions = lasso.predict(X_test_scaled)
MSE = mean_squared_error(y_test_scaled, predictions)
r2 = lasso.score(X_test_scaled, y_test_scaled)
print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.6036839199134729, R2: 0.3475808374998869


In [ ]:
plt.scatter(predictions, y_test_scaled)

In [37]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)
predictions = ridge.predict(X_test_scaled)
MSE = mean_squared_error(y_test_scaled, predictions)
r2 = ridge.score(X_test_scaled, y_test_scaled)
print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.592536660599208, R2: 0.3596280120991818


In [ ]:
plt.scatter(predictions, y_test_scaled)

In [38]:
from sklearn.linear_model import ElasticNet
elasticnet = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)
predictions = elasticnet.predict(X_test_scaled)
MSE = mean_squared_error(y_test_scaled, predictions)
r2 = elasticnet.score(X_test_scaled, y_test_scaled)
print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.5958162261028629, R2: 0.35608368814308655


In [ ]:
plt.scatter(predictions, y_test_scaled)

In [40]:
del noprice['price']
noprice.head()

,points,variety_count,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Bosnia and Herzegovina,country_Brazil,country_Bulgaria,country_Canada,...,variety_Xinomavro,variety_Xynisteri,variety_Yapincak,variety_Zibibbo,variety_Zierfandler,variety_Zierfandler-Rotgipfler,variety_Zinfandel,variety_Zlahtina,variety_Zweigelt,variety_Çalkarası
0,87,2354,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,87,117,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,86,1025,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,86,8935,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32,86,2354,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
np_scaled = X_scaler.transform(noprice)


/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [42]:
predictions_elastic = elasticnet.predict(np_scaled)
predictions_lasso = lasso.predict(np_scaled)
predictions_ridge = ridge.predict(np_scaled)

In [ ]:
df3 = pd.read_csv('wine_data.csv')
df3 = df3[["country", "points", "province", "variety", "price"]]

In [ ]:
df3 = df3[pd.isnull(df3['price'])]
df3.head

In [ ]:
df3.head()

In [ ]:
df3_noprice= df3.drop(columns=["price"])
df3_noprice.head()

In [ ]:
df3_new = pd.get_dummies(df3_noprice)
df3_new.head()

In [ ]:
df3_scaled = X_scaler.transform(df3_new)
df3_scaled

In [ ]:
len(df3_new.columns)

In [ ]:
list(set(list(X_train.columns))- set(list(df3_new.columns)))

In [ ]:
len(X_train.columns)

In [ ]:
predictions3 = model.predict(df3_new)